In [1]:
from openai import OpenAI
import json
import csv
import re
from lcpartition import *
import os
# import dotenv
import ast
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
# from validationClass3 import *

c:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\.yellowvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\.yellowvenv\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\.yellowvenv\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\tykun\OneDrive\Documents\SchoolDocs\VSCodeProjects\.yellowvenv\Lib\site-packages\transformers\utils\generi

In [2]:
#Converts to json to string
def convertToString(path): 
    with open(path, 'r') as file:
        fullText = json.load(file)
    return fullText


def listToString(list):
    str = ""
    for x in list:
        str += '\n' + x
    return str


def separateNames(page, titleIndices):
    cleanedPage = ""
    for index, content in enumerate(page):
        # Check if the current index is in titleIndices to add a newline
        if index in titleIndices:
            cleanedPage += '\n'  # Add a newline before the title/content
            cleanedPage += ' '
        cleanedPage += content
        cleanedPage += ' '
    return cleanedPage


def splitPage(page, titleIndices):
    midpoint = titleIndices[(len(titleIndices)) // 2]
    partition1 = page[0:midpoint]
    partition2 = page[midpoint-2:]
    return partition1, partition2


#Uses spacy to detect the named entities on the page and labels them
def tagNames1(rawData, titleIndices):
    names = []  # This will store the extracted names
    counter = 0
    for text in rawData:
      if counter in titleIndices:
        names.append(rawData[counter:counter+2])
      counter+=1
    return names

def outputToList(output):
    strippedOutput = output.rstrip(" \"\"\"")
    list = strippedOutput.strip("[")
    list = list.strip("]")
    list = list.split(",")
    return list

In [3]:
client = OpenAI()

#Currently using json instead of text files for the input
OCRPath = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\yellowBookStuff\\fileReads\\15pages_winter_2009.json"
filePath = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\yellowBookStuff\\fileReads\\15pages_winter_2009.json"
titleIndices = []
blockedList = []
rawData = pathToString(filePath)
singlePage = rawData[4]
fullresponse = ""

In [4]:
#This file does not run in a loop and clean every page in the text, it only takes one. Used for testing.
processedPath =  convertToString(OCRPath)
current_page = processedPath[4]
processedPathStr = str(processedPath[4])
print("this is the string we are using right now: ", processedPathStr)

this is the string we are using right now:  ['Professional Schools and Colleges', '', 'College of Business', '', 'Dean (Interim) William M. Hall........ eens beens (687) 871-2124', 'E-mail: fhall@alfred.edu', 'Education: SUNY (Genesco) BA; Syracuse MA, PhD', '', 'New York State College of Ceramics _', '', 'Dean, School of Art and Design Joseph S. Lewis Ill. ... . . (607) 871-2412', 'E-mail: lewis@alfred.edu , ,', 'Education: Hamilton BA; Maryland Inst Art MFA', '', 'Dean, School of Engineering Alastair Cormack......... (607) 871-2422', 'E-mail: cormack@alfred.edu', 'Education: Cambridge (UK) BA; U Wales (UK) MS, PhD', '', 'College of Liberal Arts and Sciences', '', 'Dean Mary McGee... 2... cece ee en eens (607) 871-2171', 'E-mail: mcgee@alfred.cdu', 'Education: Georgetown 1975 BA; Harvard', '', 'The Graduate School', 'Director Alastair Cormack............... eee eee « « (607) 871-2422', '', 'Board of Trustees', '', 'Gene M. Bernstein... 0... 0... cee nee da. Trustee', 'Affiliation: Pri

In [5]:
for line in current_page:
    print(line)

Professional Schools and Colleges

College of Business

Dean (Interim) William M. Hall........ eens beens (687) 871-2124
E-mail: fhall@alfred.edu
Education: SUNY (Genesco) BA; Syracuse MA, PhD

New York State College of Ceramics _

Dean, School of Art and Design Joseph S. Lewis Ill. ... . . (607) 871-2412
E-mail: lewis@alfred.edu , ,
Education: Hamilton BA; Maryland Inst Art MFA

Dean, School of Engineering Alastair Cormack......... (607) 871-2422
E-mail: cormack@alfred.edu
Education: Cambridge (UK) BA; U Wales (UK) MS, PhD

College of Liberal Arts and Sciences

Dean Mary McGee... 2... cece ee en eens (607) 871-2171
E-mail: mcgee@alfred.cdu
Education: Georgetown 1975 BA; Harvard

The Graduate School
Director Alastair Cormack............... eee eee « « (607) 871-2422

Board of Trustees

Gene M. Bernstein... 0... 0... cee nee da. Trustee
Affiliation: Principal, Northville Industries . a

_ 25 Melville Park Road, Melville, NY 11747
Tel: (631) 753-4211
Education: Alfred 1969 . . . .

Miche

In [6]:
import spacy
import spacy_curated_transformers
import spacy_transformers
import en_core_web_trf

#sm is only very slighly worse than trf and lg but is much faster
nlp = spacy.load("en_core_web_sm")


In [7]:
def clean_data(data):
    # List of unwanted substrings, case insensitive
    unwanted_substrings = ["education:", "affiliation:", "career:", "e-mail:", "tel:", "fax:", "internet:"]

    # Filter the list
    filtered_data = [
        element for element in data
        if element.strip() and  # Check if the element is not just whitespace
        not any(substring.lower() in element.lower() for substring in unwanted_substrings)
    ]

    return filtered_data

# Clean the data
cleaned_list = clean_data(current_page)

# Output the cleaned list
print("Cleaned Data List:")
for item in cleaned_list:
    print(item)

Cleaned Data List:
Professional Schools and Colleges
College of Business
Dean (Interim) William M. Hall........ eens beens (687) 871-2124
New York State College of Ceramics _
Dean, School of Art and Design Joseph S. Lewis Ill. ... . . (607) 871-2412
Dean, School of Engineering Alastair Cormack......... (607) 871-2422
College of Liberal Arts and Sciences
Dean Mary McGee... 2... cece ee en eens (607) 871-2171
The Graduate School
Director Alastair Cormack............... eee eee « « (607) 871-2422
Board of Trustees
Gene M. Bernstein... 0... 0... cee nee da. Trustee
_ 25 Melville Park Road, Melville, NY 11747
Michele R. Cohen .......-- eee e cere ee ee eens rn Trustee
Gregory R. COMMOFS .. 0.6... eee eee eee eee eee '.... Trustee
3445 Winton Place, Suite 112, Rochester, NY. 14623
E Peter CUNEO... 6. cece eee eect eee tenn ene Chairman
417 Fifth Avenue, 11th Floor, New York, NY 10016
Robert B. Daggett ....--. eee c cece c eee e teens Trustee
One Magnetic’ Way, Smethport, PA 16749
Terry S. Ga

In [8]:
def has_three_repeats(line):
    counter = 0
    for char in line:
        if char == '.':
            counter+=1
        if counter >= 3:
            return True
    return False



def add_guarantees(name_list):
    for line in cleaned_list:
        if has_three_repeats(line) is True and line not in name_list:
            name_list.append(line)
    return name_list


def add_only_guarantees(name_list):
    newlist = []
    for line in name_list:
        if has_three_repeats(line) is True and line not in newlist:
            newlist.append(line)
    return newlist

def remove_substrings(name_list):
    removelist = ["Dean", "JD", "Assistant Dean", "Treasurer", "Admissions", "Controller", "Emeritus", "N/A", "college", "College", "University", "university"]
    # Create a new list where each name has the specified substrings removed
    filtered_names = []
    for name in name_list:
        modified_name = name
        for substring in removelist:
            modified_name = modified_name.replace(substring, "")
        filtered_names.append(modified_name.strip())  # Remove any leading/trailing whitespace left after replacements
    return filtered_names

def add_vacancies(name_list, rawData):
    for line in rawData:
        if "Vacant" in line and line not in name_list:
            name_list.append(line)
    return name_list


In [9]:
import spacy
from spacy.pipeline import EntityRuler


name_list = []

def tag_names(cleaned_list):
    for text in cleaned_list:
        doc = nlp(text)
        for ent in doc.ents:
            if ent.label_ == "PERSON" and text not in name_list:
                name_list.append(text)
    return name_list


def remove_mistakes(name_list):
    removelist = ["Dean", "JD", "Assistant Dean", "Treasurer", "Admissions", "Controller", "Emeritus", "N/A", "", "college", "College", "University", "university"]
    removeset = set(removelist)
    filtered_names = [name for name in name_list if name not in removeset]
    return filtered_names



nlp = spacy.load("en_core_web_sm")
name_list = tag_names(cleaned_list)
nlp = spacy.load("en_core_web_trf")
name_list = tag_names(cleaned_list)
nlp = spacy.load("en_core_web_lg")
name_list = tag_names(cleaned_list)
name_list = remove_mistakes(name_list)
name_list = add_guarantees(name_list)
name_list = add_vacancies(name_list, cleaned_list)

print("\nAll texts containing a person:")
for x in name_list:
    print(x)


All texts containing a person:
Dean (Interim) William M. Hall........ eens beens (687) 871-2124
Dean Mary McGee... 2... cece ee en eens (607) 871-2171
Director Alastair Cormack............... eee eee « « (607) 871-2422
Gene M. Bernstein... 0... 0... cee nee da. Trustee
_ 25 Melville Park Road, Melville, NY 11747
Michele R. Cohen .......-- eee e cere ee ee eens rn Trustee
3445 Winton Place, Suite 112, Rochester, NY. 14623
E Peter CUNEO... 6. cece eee eect eee tenn ene Chairman
Robert B. Daggett ....--. eee c cece c eee e teens Trustee
Terry S. Galanis, Jr... .- eee e eee ees Trustee
Leslie J. Gelber... 6.6 eee eect e ee cee eee Vice Chairman
Frederick A. G@Orge «6... cere rete eee cece e eee ees beeen Trustee
Frederick W. GiDbDS .... 6-60 e eee eee cece eee erect es Trustee
John S. Gilbertson .......-- 00. eee eee eee ence ees Trustee
Jay L. Haberland........--> eee eee ees vane nee e eee ee Trustee
Stephen K. Heine ....---------- bee eee ees ence eens cee Trustee
Thomas R. Hinman.....

In [10]:
#Iterate through the name list and tell gpt to detect the name from every line
verified_name_list = []

for line in name_list:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You have been assigned the role of a data extractor. Your task is to process a line of text and extract the person's name that appears \
             within that line. Sometimes a line of text will not contain a person's name. In this case, only return the string: \"N/A\". Otherwise, only return the person's name\
             that appears in the line of text."},
            {"role": "system", "content": "If you decide to return \"N/A\", be positive that the line did not contain a person's name. Otherwise, return only their name."},
            {"role": "system", "content": "The name should always be a full name. Never only return a first name. It will always be a first and last name."},
            {"role": "system", "content": "Names can also appear in a format where the first name is abbreviated, but their middle and surname are not. For example, a possible name that can appear is: \"M. Parker Cook\""},
            {"role": "system", "content": "EVERY line that contains a phone number will contain a name. Never put \"N/A\" if the line contains a phone number. Also, every line that contains long strings of periods (\".\") will contain a name on that line."},
            {"role": "system", "content": "Also, for this task, consider the word \"Vacannt\" to be a name."},
            {"role": "system", "content": "Do not add any extraneous text to your response. Doing so will break the pipeline."},
            {"role": "user", "content": line},
        ],
        temperature=.1,
        frequency_penalty=0,
        presence_penalty=0,
        # seed=1001
    )

    cleaned_name = response.choices[0].message.content
    verified_name_list.append(str(cleaned_name))

print("This is the cleaned list of names: ")
print(verified_name_list)

This is the cleaned list of names: 
['William M. Hall', 'Mary McGee', 'Alastair Cormack', 'Gene M. Bernstein', 'N/A', 'Michele R. Cohen', 'N/A', 'E Peter CUNEO', 'Robert B. Daggett', 'Terry S. Galanis', 'Leslie J. Gelber', 'Frederick A. G@Orge', 'Frederick W. GiDbDS', 'John S. Gilbertson', 'Jay L. Haberland', 'Stephen K. Heine', 'Thomas R. Hinman', 'James M. Jordan', 'Charles P Joyce', 'Elizabeth A. Judson', 'Christine P Kulp', 'Jeffrey S. Maurer', 'Robert R. MicComsey', 'Robert K. Meltzer', 'Victoria A. Meyer', 'Terry A. Michalske', 'N/A', 'David J. Miller', 'Marlin Miller', 'Joel P Moskowitz', 'N/A', 'Richard W. Mott', 'N/A', 'Albert E. Paladino', 'Robert H. Perlman', 'William Pullman', 'N/A', 'Kathleen A. Richardson', 'Lance T. Shaner', 'Joseph S. Lewis', 'Alastair Cormack', 'Gregory R. COMMOFS', 'N/A']


In [11]:
import re
from fuzzywuzzy import process, fuzz


#I did not write any of these functions

def normalize_text(text):
    # Lowercase and remove non-alphanumeric characters except spaces
    return re.sub(r'[^a-z0-9\s]', '', text.lower())



def find_matches(names, dataset, threshold=85):
    normalized_dataset = normalize_text(dataset)
    words = normalized_dataset.split()
    segments = [' '.join(words[i:i + 12]) for i in range(len(words) - 11)]
    # print(segments)

    matched_strings = []

    for name in names:
        normalized_string = normalize_text(name)
        # Use fuzzy matching to find the best segment that matches the string
        best_matches = process.extractOne(normalized_string, segments, scorer=fuzz.partial_ratio, score_cutoff=threshold)

        if best_matches:
            best_segment, match_score = best_matches
            segment_index = segments.index(best_segment)
            # Append matched string along with its first found index for sorting purposes
            matched_strings.append((name, segment_index))

    # Sort matched strings by their first appearance index in the dataset
    matched_strings.sort(key=lambda x: x[1])

    # Extract only the string names for final output, omitting the index
    sorted_matched_strings = [match[0] for match in matched_strings]
    return sorted_matched_strings



def split_duplicates(names, dataset, threshold = 85):
    indices = {}
    start = 0
    end = 1
    #go through whole list of names
    while(end < len(names)):
        #if the next name is the same as the current name
        linecounter = 0
        # if(names[end] == names[start]):
        if(fuzz.partial_ratio(names[end], names[start]) >= 90):
            #Go through the dataset until you find the second occurrence of this name:
            count = 0
            for line in dataset:
                if fuzz.partial_ratio(line, names[start]) >= threshold:
                    # print(line, "    ", names[start])
                    if(count > 0):
                        indices[names[start] + " 2"] = linecounter
                        break
                    count+=1
                linecounter+=1
        else: 
            for line in dataset:
                if fuzz.partial_ratio(line, names[start]) >= threshold:
                    indices[names[start]] = linecounter
                    break
                linecounter+=1

        start+=1
        end += 1

    if end == len(names):
        linecounter = 0
        for line in dataset:
            if fuzz.partial_ratio(line, names[start]) >= threshold:
                indices[names[start]] = linecounter
                break
            linecounter += 1
            
    return indices

                
def sort_names(names):
     # Sort the dictionary by value in ascending order and return the keys
    sorted_names = sorted(names.items(), key=lambda item: item[1])
    # Extract the keys from the sorted items
    sorted_name_list = [name for name, index in sorted_names]
    return sorted_name_list




def remove_duplicates(input_list):
    seen = set()
    unique_list = []
    for item in input_list:
        if item not in seen:
            unique_list.append(item)
            seen.add(item)
    return unique_list



def find_non_substring_elements(larger_list, smaller_list, threshold=80):
    non_substrings = []
    
    # Check each element in the larger list
    for large_element in larger_list:
        found = False
        # Check if any element in the smaller list is a fuzzy substring of the current element in the larger list
        for element in smaller_list:
            if fuzz.partial_ratio(element.lower(), large_element.lower()) >= threshold:
                found = True
                break
        # If no elements from the smaller list match as substrings under the fuzzy conditions, add the large element to the result list
        if not found:
            non_substrings.append(large_element)

    return non_substrings


In [12]:
#Clean the output of names:
# verified_name_list = remove_mistakes(verified_name_list)
verified_name_list = remove_substrings(verified_name_list)
verified_name_list = find_matches(verified_name_list, processedPathStr)

unaltered_names  = verified_name_list






print("This is the cleaned name list: ")
for name in verified_name_list:
    print(name)


print("\n\n\n")

# #For duplicate names, we need some way to detect the second occurrence
# name_indices = split_duplicates(verified_name_list, processedPath[1])


# for name, index in name_indices.items():
#     print(f"{name}: {index}")

#Probably do not want to remove duplicates because people actually appear multiple times
# verified_name_list = remove_duplicates(verified_name_list

This is the cleaned name list: 
William M. Hall
Joseph S. Lewis
Alastair Cormack
Alastair Cormack
Mary McGee
Gene M. Bernstein
Michele R. Cohen
Gregory R. COMMOFS
E Peter CUNEO
Robert B. Daggett
Terry S. Galanis
Leslie J. Gelber
Frederick A. G@Orge
Frederick W. GiDbDS
John S. Gilbertson
Jay L. Haberland
Stephen K. Heine
Thomas R. Hinman
James M. Jordan
Charles P Joyce
Elizabeth A. Judson
Christine P Kulp
Jeffrey S. Maurer
Robert R. MicComsey
Robert K. Meltzer
Victoria A. Meyer
Terry A. Michalske
David J. Miller
Marlin Miller
Joel P Moskowitz
Richard W. Mott
Robert H. Perlman
William Pullman
Kathleen A. Richardson
Lance T. Shaner






In [13]:
disregarded_elements = find_non_substring_elements(name_list, verified_name_list)
possible_elements = add_only_guarantees(disregarded_elements)

for name in possible_elements:
    print(name)

Albert E. “Al” Paladino... 0.0... 0000... e ccc eee ees Trustee


In [14]:
#Iterate through the name list and tell gpt to detect the name from every line
for line in possible_elements:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You have been assigned the role of a data extractor. Your task is to process a line of text and extract the person's FULL name \
             within that line. There should always be a name in the line, however, if you are SURE that there isn't, then return: \"N/A\"."},
             {"role": "system", "content": "EVERY line that contains 3 or more periods will contain a name. In this case, the name will appear before the periods do. If you "},
            {"role": "system", "content": "EVERY line that contains a phone number will contain a name. Never put \"N/A\" if the line contains a phone number. Also, every line that contains long strings of periods (\".\") will contain a name on that line."},
            {"role": "system", "content": "Also, for this task, consider the word \"Vacannt\" to be a name."},
            {"role": "system", "content": "Do not add any extraneous text to your response. Doing so will break the pipeline."},
            {"role": "user", "content": line},
        ],
        temperature=.1,
        frequency_penalty=0,
        presence_penalty=0,
        # seed=1001
    )
    cleaned_name = response.choices[0].message.content
    print(cleaned_name)
    verified_name_list.append(str(cleaned_name))

Albert E. “Al” Paladino


In [15]:
#Clean the output of names:
verified_name_list = remove_mistakes(verified_name_list)
verified_name_list = remove_substrings(verified_name_list)
verified_name_list = find_matches(verified_name_list, processedPathStr)

#Probably do not want to remove duplicates because people actually appear multiple times
# verified_name_list = remove_duplicates(verified_name_list)
#Sort the names with the duplicate taken care of
#For duplicate names, we need some way to detect the second occurrence
name_indices = split_duplicates(verified_name_list, current_page)
sorted_names = sort_names(name_indices)
verified_name_list = sorted_names


print("This is the cleaned name list: ")
for name in verified_name_list:
    print(name)

This is the cleaned name list: 
William M. Hall
Joseph S. Lewis
Alastair Cormack
Mary McGee
Alastair Cormack 2
Gene M. Bernstein
Michele R. Cohen
Gregory R. COMMOFS
E Peter CUNEO
Robert B. Daggett
Terry S. Galanis
Leslie J. Gelber
Frederick A. G@Orge
Frederick W. GiDbDS
John S. Gilbertson
Jay L. Haberland
Stephen K. Heine
Thomas R. Hinman
James M. Jordan
Charles P Joyce
Elizabeth A. Judson
Christine P Kulp
Jeffrey S. Maurer
Robert R. MicComsey
Robert K. Meltzer
Victoria A. Meyer
Terry A. Michalske
David J. Miller
Marlin Miller
Joel P Moskowitz
Richard W. Mott
Albert E. “Al” Paladino
Robert H. Perlman
William Pullman
Kathleen A. Richardson
Lance T. Shaner


In [16]:
import spacy
import json
import nltk
from spacy import displacy
import re
from fuzzywuzzy import process


'''
Currently this cannot account for the situation where there's a title that starts on the line above a name, but also leaks into the line with the name
SpaCy doesn't detect every name, so this ends up with blocks of multiple names somtimes
'''

#Words that appear in titles
nlp = spacy.load("en_core_web_sm")
TITLE_BANK = ["assistant", "associate", "chief", "command", "controller", "counsel", "director", "dean", "deputy", "director", "executive", "general", "officer", "president", "provost", "registrar", "treasurer", "trustee", "vice"]


#Uses spacy to detect the named entities on the page and labels them
def tagNames(rawData, titleIndices, nlp):
    lineNumber = 0
    nameLines = []
    for text in rawData:
        doc = nlp(text)
        for ent in doc.ents:
            isPerson = str(ent.label_) == "PERSON"
            #Add each line that contains a person into the nameLines list
            if isPerson:
                nameLines.append(text)

            isGPE = any(ent.label_ == "GPE" for ent in doc.ents)
            #Will have to add a different way to handle the word dean, just a temporary solution
            isDean = "Dean" in str(ent.text)
            noKeywords = all(keyword not in text for keyword in ["Education", "Affiliation", "Career:"])
            #Assign the next line, ensure that it's not out of bound. Also check two lines ahead
            #-------------------------------------------------------------------------------
            if lineNumber < len(rawData) - 1:
                nextLine = rawData[lineNumber+1]
            else:
                nextLine = rawData[0]

            if lineNumber < len(rawData) - 2:
                twoLines = rawData[lineNumber+2]
            else:
                twoLines = rawData[0]
            #-------------------------------------------------------------------------------
            
            noTel = "tel:" not in nextLine.lower() and "tel" not in twoLines.lower()

            if isPerson and noKeywords and not isDean and noTel:
                # print(str(ent), '\n')
                titleIndices = checkStructure(text, rawData[lineNumber-1], str(ent.text), lineNumber, rawData, nameLines, titleIndices)
        lineNumber+=1 
    return titleIndices



def new_tagNames1(rawData, nameList, titleIndices):
    lineNumber = 0
    nameCounter = 0
    correctName = ""
    nameLines = []
    for text in rawData:
        found = False
        for name in nameList:
            if name in text and "Affiliation" not in text:
                # print(name, "   ",  text)
                found = True
                correctName = name
                break

        if found:
            nameLines.append(text)  # Append the text where the name matched
            if lineNumber < len(rawData) - 1:
                nextLine = rawData[lineNumber + 1]
            else:
                nextLine = rawData[0]

            if lineNumber < len(rawData) - 2:
                twoLines = rawData[lineNumber + 2]
            else:
                twoLines = rawData[0]

            previousLine = rawData[lineNumber - 1] if lineNumber > 0 else rawData[-1]
            titleIndices = checkStructure1(text, previousLine, correctName, lineNumber, rawData, nameLines, titleIndices)
            nameCounter += 1
        
        lineNumber += 1
    return titleIndices

def find_misses(rawData, nameList, titleIndices):
    lineNumber = 0
    for line in rawData:
        line = line.lower()
        title_on_line = False
        for x in TITLE_BANK:
             if x in line:
                 title_on_line = True
        enoughDots = has_three_repeats(line)
        if enoughDots and title_on_line and lineNumber not in titleIndices:
            
            titleIndices.append(lineNumber)
            print("This line was used: ", line)
            print("line at the index: ", singlePage[lineNumber])
        lineNumber += 1
    return titleIndices







#Given a line of text, check to see what the name/title layout is
def checkStructure1(currentLine, previousLine, name, lineNumber, rawData, nameLines, titleIndices):
    nameStartIndex = currentLine.index(name)
    nameEndIndex = nameStartIndex + len(name)
    previousContainsTitle = checkTitleBank(previousLine)
    if(currentLine[nameEndIndex:] != "" and currentLine[:nameStartIndex] != ""):
        #Check for case where the title is above the name and on the same line. This is not a perfect check, but it should overcount instead of undercount
        #Check that the previousline wasn't tagged as a name line and that it doesn't contain a colon. (Education: , E-mail:, etc)
        if(":" not in previousLine and previousLine not in nameLines and previousContainsTitle):
            titleIndices.append(lineNumber-1)
        else:
            titleIndices.append(lineNumber)
    elif(currentLine[:nameStartIndex] == "" and currentLine[-2].isdigit() and previousContainsTitle):
        titleIndices.append(lineNumber-1)
    elif(currentLine[:nameStartIndex] == "" and currentLine[nameEndIndex:] != ""):
        titleIndices.append(lineNumber)
    else:
        titleIndices.append(lineNumber)
    return titleIndices



#

In [17]:
titleIndices = []
singlePage = removeEmptyElements(singlePage)
singlePage = removeEmptyElements(singlePage)
titleIndices = new_tagNames1(singlePage,unaltered_names, titleIndices)
titleIndices = find_misses(singlePage, unaltered_names, titleIndices)
# titleIndices = breakChunks(singlePage, titleIndices)
# titleIndices = breakChunks(singlePage, titleIndices)
# titleIndices = breakDoubleNames(singlePage, titleIndices)
# titleIndices = breakTwoLineDoubleName(singlePage, titleIndices)
titleIndices = remove_duplicates(titleIndices)
titleIndices = sorted(titleIndices)
titleIndices = connectSingleLines(singlePage, titleIndices)
print(titleIndices)
printFormattedText(singlePage, titleIndices)

[2, 6, 9, 13, 17, 19, 24, 25, 30, 35, 40, 44, 50, 54, 59, 64, 66, 68, 72, 77, 81, 84, 87, 91, 93, 98, 102, 109, 115, 119, 124, 128, 131, 133, 136, 141]
Professional Schools and Colleges
College of Business



Dean (Interim) William M. Hall........ eens beens (687) 871-2124
E-mail: fhall@alfred.edu
Education: SUNY (Genesco) BA; Syracuse MA, PhD
New York State College of Ceramics _



Dean, School of Art and Design Joseph S. Lewis Ill. ... . . (607) 871-2412
E-mail: lewis@alfred.edu , ,
Education: Hamilton BA; Maryland Inst Art MFA



Dean, School of Engineering Alastair Cormack......... (607) 871-2422
E-mail: cormack@alfred.edu
Education: Cambridge (UK) BA; U Wales (UK) MS, PhD
College of Liberal Arts and Sciences



Dean Mary McGee... 2... cece ee en eens (607) 871-2171
E-mail: mcgee@alfred.cdu
Education: Georgetown 1975 BA; Harvard
The Graduate School



Director Alastair Cormack............... eee eee « « (607) 871-2422
Board of Trustees



Gene M. Bernstein... 0... 0... cee nee da. 

In [18]:
print(titleIndices)
print("Length of title indices : ", len(titleIndices))
print("length of name list: ", len(verified_name_list))

[2, 6, 9, 13, 17, 19, 24, 25, 30, 35, 40, 44, 50, 54, 59, 64, 66, 68, 72, 77, 81, 84, 87, 91, 93, 98, 102, 109, 115, 119, 124, 128, 131, 133, 136, 141]
Length of title indices :  36
length of name list:  36


In [19]:
def list_to_keys(input_list):
    # Create a dictionary with list elements as keys and None as values
    result_dict = {item: None for item in input_list}
    return result_dict


# def list_to_keys(input_list):
#     # Create a dictionary with list elements as keys and empty lists as values
#     result_dict = {item: [] for item in input_list}
#     return result_dict

name_dict=  list_to_keys(verified_name_list)
print(name_dict)

{'William M. Hall': None, 'Joseph S. Lewis': None, 'Alastair Cormack': None, 'Mary McGee': None, 'Alastair Cormack 2': None, 'Gene M. Bernstein': None, 'Michele R. Cohen': None, 'Gregory R. COMMOFS': None, 'E Peter CUNEO': None, 'Robert B. Daggett': None, 'Terry S. Galanis': None, 'Leslie J. Gelber': None, 'Frederick A. G@Orge': None, 'Frederick W. GiDbDS': None, 'John S. Gilbertson': None, 'Jay L. Haberland': None, 'Stephen K. Heine': None, 'Thomas R. Hinman': None, 'James M. Jordan': None, 'Charles P Joyce': None, 'Elizabeth A. Judson': None, 'Christine P Kulp': None, 'Jeffrey S. Maurer': None, 'Robert R. MicComsey': None, 'Robert K. Meltzer': None, 'Victoria A. Meyer': None, 'Terry A. Michalske': None, 'David J. Miller': None, 'Marlin Miller': None, 'Joel P Moskowitz': None, 'Richard W. Mott': None, 'Albert E. “Al” Paladino': None, 'Robert H. Perlman': None, 'William Pullman': None, 'Kathleen A. Richardson': None, 'Lance T. Shaner': None}


In [20]:
# Keys for each dictionary
keys = ['name', 'title', 'education', 'affiliation']

# Check if we have enough names for the dictionaries
if len(verified_name_list) < len(keys):
    raise ValueError("Not enough names provided")

# Creating a list of dictionaries, initializing 'name' with a name from the list
list_of_dicts = [{**{key: 'N/A' for key in keys}, 'name': name} for name in verified_name_list]

# Printing the list of dictionaries
for i, d in enumerate(list_of_dicts):
    print(f"Dictionary {i+1}: {d}")

Dictionary 1: {'name': 'William M. Hall', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 2: {'name': 'Joseph S. Lewis', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 3: {'name': 'Alastair Cormack', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 4: {'name': 'Mary McGee', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 5: {'name': 'Alastair Cormack 2', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 6: {'name': 'Gene M. Bernstein', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 7: {'name': 'Michele R. Cohen', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 8: {'name': 'Gregory R. COMMOFS', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 9: {'name': 'E Peter CUNEO', 'title': 'N/A', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 10: {'name': 'Robert B. Daggett', 'title': 'N/A', 'education': 'N/A', 'affiliation': '

In [21]:
#Iterate through the name list and tell gpt to detect the name from every line
start = 0
end = 1
counter =  0

for name in verified_name_list:
    if(end < len(titleIndices)):
        if name in str(singlePage[titleIndices[start]:titleIndices[start] + 1]):
            textBlock = singlePage[titleIndices[start]:titleIndices[start] + 1]
        else: 
            textBlock = singlePage[titleIndices[start]:titleIndices[start] + 2]
    else:
        textBlock = singlePage[titleIndices[start]:]
    start += 1
    end += 1
    print(name, "    " , textBlock)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You have been assigned the role of a data extractor. Your task is to process a block of text and extract the person's FULL job title \
             within that text. You will be provided with the person's name to extract for, and a block of text containing that person's name and some information about them.\
             note that the job title must either appear on the same line as the person's name, or before it. It will never appear on the line after."},
            {"role": "system", "content": "Return only the job title. Do NOT return the person's name. Also do not return any extraneous text. Doing so will break the pipeline."},
            {"role": "system", "content": "Pleae ensure to return the full job title. For example, if the string: \"Dean, Student Relations\" were to appear, return the \
             entire string as the title, instead of only Dean."},
            {"role": "system", "content": "In cases where the text contains ambiguities or conflicting information, use your best judgement to resolve them and \
            extract the most accurate and relevant information."},
            {"role": "system", "content": "There should ALWAYS be a relevant title within the supplied text. If you cannot determine a suitable title, you must user your best judgement. \
             to return an appropriate title. As a rule of thumb, there's a good chance that the word \"Trustee\" was misspelled, and is the correct title to return."},
            {"role": "system", "content": "You must return the job title and only the job title. Do not return any other text, or write anything else in the message. If you cannot determine a title, return only: \"N/A\""},
            {"role": "user", "content": str(name)},
            {"role": "user", "content": "Here is the text block: "},
            {"role": "user", "content": str(textBlock)},
        ],
        temperature=.1,
        frequency_penalty=0,
        presence_penalty=0,
        # seed=1001
    )
    job_title = response.choices[0].message.content.strip()  # Assume the content is the job title cleaned
    # Store the job title in the dictionary with the name as the key
    name_dict[name] = job_title
    list_of_dicts[counter]['title'] = job_title
    counter+=1

William M. Hall      ['Dean (Interim) William M. Hall........ eens beens (687) 871-2124']
Joseph S. Lewis      ['Dean, School of Art and Design Joseph S. Lewis Ill. ... . . (607) 871-2412']
Alastair Cormack      ['Dean, School of Engineering Alastair Cormack......... (607) 871-2422']
Mary McGee      ['Dean Mary McGee... 2... cece ee en eens (607) 871-2171']
Alastair Cormack 2      ['Director Alastair Cormack............... eee eee « « (607) 871-2422', 'Board of Trustees']
Gene M. Bernstein      ['Gene M. Bernstein... 0... 0... cee nee da. Trustee']
Michele R. Cohen      ['Michele R. Cohen .......-- eee e cere ee ee eens rn Trustee']
Gregory R. COMMOFS      ["Gregory R. COMMOFS .. 0.6... eee eee eee eee eee '.... Trustee"]
E Peter CUNEO      ['E Peter CUNEO... 6. cece eee eect eee tenn ene Chairman']
Robert B. Daggett      ['Robert B. Daggett ....--. eee c cece c eee e teens Trustee']
Terry S. Galanis      ['Terry S. Galanis, Jr... .- eee e eee ees Trustee']
Leslie J. Gelber      ['Lesl

In [22]:
# Printing the list of dictionaries
for i, d in enumerate(list_of_dicts):
    print(f"Dictionary {i+1}: {d}")

Dictionary 1: {'name': 'William M. Hall', 'title': 'Interim Dean', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 2: {'name': 'Joseph S. Lewis', 'title': 'Dean, School of Art and Design', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 3: {'name': 'Alastair Cormack', 'title': 'Dean, School of Engineering', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 4: {'name': 'Mary McGee', 'title': 'Dean', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 5: {'name': 'Alastair Cormack 2', 'title': 'Director', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 6: {'name': 'Gene M. Bernstein', 'title': 'Trustee', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 7: {'name': 'Michele R. Cohen', 'title': 'Trustee', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 8: {'name': 'Gregory R. COMMOFS', 'title': 'Trustee', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 9: {'name': 'E Peter CUNEO', 'title': 'Chairman', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 10:

In [23]:

def affiliation_education(singlePage, titleIndices, verified_name_list, keyword, column):
    name_dict = {}
    start = 0
    end = 1
    counter = 0
    # keyword = "affiliation"
    threshold = 80

    for name in verified_name_list:
        if end < len(titleIndices):
            textBlock = singlePage[titleIndices[start]:titleIndices[start + 1]]
        else:
            textBlock = singlePage[titleIndices[start]:]
        start += 1
        end += 1

        # Normalize text block to lowercase for consistency in matching
        textBlock = str(textBlock).lower()
        print(textBlock)

        # Search for a similar substring using fuzzy matching
        # if fuzz.partial_ratio(textBlock, keyword) >= threshold:
        if keyword in textBlock:
            print(name, "   ", keyword)
            start_index = textBlock.find(keyword) + len(keyword) + 1  # Adjust to start after the keyword
            end_index = len(textBlock)  # Default to the end of the text block

            # Find the minimum index of any of the other keywords
            for key in ["career: ", "education: ", "email: ", "fax: ", "tel: ", "internet: "]:
                key_index = textBlock.find(key, start_index)
                if key_index != -1:
                    end_index = min(end_index, key_index)

            extracted_text = textBlock[start_index:end_index].strip()
            # Check if the name is already in the dictionary and update or set the affiliation
            # if name in name_dict:
            #     name_dict[name] += "; " + extracted_text  # Append additional affiliation with a separator if exists
            #     list_of_dicts[counter][column] = extracted_text
            # else:
            name_dict[name] = extracted_text  # Set the affiliation if not yet in the dictionary
            list_of_dicts[counter][column] = extracted_text
        elif(list_of_dicts[counter][column] == None):
            list_of_dicts[counter][column] = "N/A"
        counter += 1

        # Optionally print the text block for debugging
        # print(textBlock)

    return name_dict



new_affiliation_dict = affiliation_education(singlePage, titleIndices, verified_name_list, "affiliation:", "affiliation")
new_education_dict = affiliation_education(singlePage, titleIndices, verified_name_list, "education:", "education")
new_career_dict = affiliation_education(singlePage, titleIndices, verified_name_list, "career:", "affiliation")

print("Here is the dictionary of names and affiliations: ")
for name, affiliation in new_affiliation_dict.items():
    print(f"{name}: {affiliation}")



print("\n\n Here is the dictionary of names and education: ")
for name, education in new_education_dict.items():
    print(f"{name}: {education}")




print("\n\n Here is the dictionary of names and career: ")
for name, career in new_career_dict.items():
    print(f"{name}: {career}")


['dean (interim) william m. hall........ eens beens (687) 871-2124', 'e-mail: fhall@alfred.edu', 'education: suny (genesco) ba; syracuse ma, phd', 'new york state college of ceramics _']
['dean, school of art and design joseph s. lewis ill. ... . . (607) 871-2412', 'e-mail: lewis@alfred.edu , ,', 'education: hamilton ba; maryland inst art mfa']
['dean, school of engineering alastair cormack......... (607) 871-2422', 'e-mail: cormack@alfred.edu', 'education: cambridge (uk) ba; u wales (uk) ms, phd', 'college of liberal arts and sciences']
['dean mary mcgee... 2... cece ee en eens (607) 871-2171', 'e-mail: mcgee@alfred.cdu', 'education: georgetown 1975 ba; harvard', 'the graduate school']
['director alastair cormack............... eee eee « « (607) 871-2422', 'board of trustees']
['gene m. bernstein... 0... 0... cee nee da. trustee', 'affiliation: principal, northville industries . a', '_ 25 melville park road, melville, ny 11747', 'tel: (631) 753-4211', 'education: alfred 1969 . . . .']

In [24]:
# Printing the list of dictionaries
for i, d in enumerate(list_of_dicts):
    print(f"Dictionary {i+1}: {d}")

Dictionary 1: {'name': 'William M. Hall', 'title': 'Interim Dean', 'education': "suny (genesco) ba; syracuse ma, phd', 'new york state college of ceramics _']", 'affiliation': 'N/A'}
Dictionary 2: {'name': 'Joseph S. Lewis', 'title': 'Dean, School of Art and Design', 'education': "hamilton ba; maryland inst art mfa']", 'affiliation': 'N/A'}
Dictionary 3: {'name': 'Alastair Cormack', 'title': 'Dean, School of Engineering', 'education': "cambridge (uk) ba; u wales (uk) ms, phd', 'college of liberal arts and sciences']", 'affiliation': 'N/A'}
Dictionary 4: {'name': 'Mary McGee', 'title': 'Dean', 'education': "georgetown 1975 ba; harvard', 'the graduate school']", 'affiliation': 'N/A'}
Dictionary 5: {'name': 'Alastair Cormack 2', 'title': 'Director', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 6: {'name': 'Gene M. Bernstein', 'title': 'Trustee', 'education': "alfred 1969 . . . .']", 'affiliation': "principal, northville industries . a', '_ 25 melville park road, melville, ny 11747

In [25]:
def process_dictionaries(list_of_dicts):
    # Characters to remove from the dictionary values
    chars_to_remove = '[]\\"`|\''
    # A translation table to remove specified characters
    remove_chars_table = str.maketrans('', '', chars_to_remove)
    
    # Processed list of dictionaries
    processed_list = []

    # Iterate through each dictionary in the list
    for dictionary in list_of_dicts:
        # Create a new dictionary for the processed values
        processed_dict = {}
        # Iterate through each key-value pair in the dictionary
        for key, value in dictionary.items():
            # Check if the value is a string before processing
            if isinstance(value, str):
                # Remove unwanted characters
                cleaned_value = value.translate(remove_chars_table)
                # Capitalize the first letter of each word and update the dictionary
                processed_dict[key] = cleaned_value.title()
            else:
                # Directly assign non-string values
                processed_dict[key] = value
        # Add the processed dictionary to the list
        processed_list.append(processed_dict)

    return processed_list


list_of_dicts = process_dictionaries(list_of_dicts)

# Printing the list of dictionaries
for i, d in enumerate(list_of_dicts):
    print(f"Dictionary {i+1}: {d}")

Dictionary 1: {'name': 'William M. Hall', 'title': 'Interim Dean', 'education': 'Suny (Genesco) Ba; Syracuse Ma, Phd, New York State College Of Ceramics _', 'affiliation': 'N/A'}
Dictionary 2: {'name': 'Joseph S. Lewis', 'title': 'Dean, School Of Art And Design', 'education': 'Hamilton Ba; Maryland Inst Art Mfa', 'affiliation': 'N/A'}
Dictionary 3: {'name': 'Alastair Cormack', 'title': 'Dean, School Of Engineering', 'education': 'Cambridge (Uk) Ba; U Wales (Uk) Ms, Phd, College Of Liberal Arts And Sciences', 'affiliation': 'N/A'}
Dictionary 4: {'name': 'Mary Mcgee', 'title': 'Dean', 'education': 'Georgetown 1975 Ba; Harvard, The Graduate School', 'affiliation': 'N/A'}
Dictionary 5: {'name': 'Alastair Cormack 2', 'title': 'Director', 'education': 'N/A', 'affiliation': 'N/A'}
Dictionary 6: {'name': 'Gene M. Bernstein', 'title': 'Trustee', 'education': 'Alfred 1969 . . . .', 'affiliation': 'Principal, Northville Industries . A, _ 25 Melville Park Road, Melville, Ny 11747, '}
Dictionary 7:

In [26]:
filename = 'NER_CONTEXT_OUTPUT.csv'
# Open the file in write mode
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the values of each dictionary to the CSV file as rows
    for dictionary in list_of_dicts:
        # Extracting values in the order of the keys
        row = [dictionary[key] for key in dictionary]
        writer.writerow(row)

print(f'Data written to {filename} successfully.')

Data written to NER_CONTEXT_OUTPUT.csv successfully.


In [27]:
import nltk
import csv
import json
import pandas as pd
import chardet
import Levenshtein as lvs

#Change to whatever you need
filePathOriginal = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\SecondYearStuff\\ConnectedData\ValidationSets\\auburnO.csv"
filePathValidated = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\SecondYearStuff\\ConnectedData\ValidationSets\\auburnV.csv"
pageTitle = "auburnAccuracies"



def read_names_original(filePath,):
     # Detect file encoding
    with open(filePath, 'rb') as f:
        result = chardet.detect(f.read())
    try:
        # Read the CSV file
        df = pd.read_csv(filePath, sep='|', encoding=result['encoding'], skiprows = 1, on_bad_lines='skip')
        
        # Assuming the names or titles are in the first column
        nameColumn = df.iloc[:, 1]  # Access the first column by index
        nameString = "\n".join(nameColumn.astype(str))
        return nameString
    except Exception as e:
        print("Error: ", e)
        return ' '
    

def read_names_validated(filePath):
    # Detect file encoding
    with open(filePath, 'rb') as f:
        result = chardet.detect(f.read())
    try:
        # Read the CSV file
        df = pd.read_csv(filePath, encoding=result['encoding'], on_bad_lines='skip')

        # Assuming the names or titles are in the second column
        nameColumn = df.iloc[:, 1]  # Access the name column by index
        nameString = "\n".join(nameColumn.astype(str))
        return nameString
    except Exception as e:
        print("Error: ", e)
        return ' '

def extractAllOriginal(path):
    columnDict = {}
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
    df = pd.read_csv(path, sep = '|', encoding = result['encoding'], skiprows = 1, on_bad_lines = 'skip')
    titleColumn = df.iloc[:, 0].str.strip().tolist()
    nameColumn = df.iloc[:, 1].str.strip().tolist()
    educationColumn = df.iloc[:, 2].str.strip().tolist()
    affiliationColumn = df.iloc[:, 3].str.strip().tolist()
    # schoolColumn = df.iloc[:, 5].str.strip().tolist()
    columnDict['Title'], columnDict["Name"], columnDict["Education"], columnDict["Affiliation"] = titleColumn, nameColumn, educationColumn, affiliationColumn
    return columnDict


def extractAllValidation(filePath):
    #Goes through the entire table and stores everything in a dictionary (validation tables)
    columnDict = {}
    with open(filePath, 'rb') as f:
        result = chardet.detect(f.read())
    df = pd.read_csv(filePath, encoding=result['encoding'], skiprows = 0,on_bad_lines='skip')
    titleColumn = df.iloc[:, 0].str.strip().tolist()
    nameColumn = df.iloc[:, 1].str.strip().tolist()
    educationColumn = df.iloc[:, 2].str.strip().tolist()
    affiliationColumn = df.iloc[:, 3].str.strip().tolist()
    # schoolColumn = df.iloc[:, 5].str.strip().tolist()

    columnDict['Title'], columnDict["Name"], columnDict["Education"], columnDict["Affiliation"] = titleColumn, nameColumn, educationColumn, affiliationColumn
    return columnDict


def averageLevenshtein(distanceList, mode):
    #If supplied with a 0 for mode, use all values, if supplied with a 1, don't use outliers
    counter = 0
    summation = 0
    if mode == 0:
        for x in distanceList:
            counter+=1
            summation+=x
    elif mode == 1:
        for x in distanceList:
            if x != 0:
                counter+=1
                summation+=x
    else:
        print("Invalid mode selected. Choose a 0 or 1.")
        return
    average = summation/counter
    return average


def tableCompare(original, validated, weight):
    #A weight value of zero will weigh every column the same, while 1 will put a larger emphasis on the name, title, and school
    #do not want to look at this function ever again
    distanceListT, distanceListN, distanceListE, distanceListA = [], [], [], []
    originalTitles, originalNames, originalEducation, originalAffiliation = original["Title"], original["Name"], original["Education"], original["Affiliation"]
    validatedTitles, validatedNames, validatedEducation, validatedAffiliation = validated["Title"], validated["Name"], validated["Education"], validated["Affiliation"]
    lengthdif = len(validatedNames) - len(originalNames)
    counterT, counterN, counterE, counterA, counterS, counterX = 0, 0, 0, 0, 0, 0
    
    if(lengthdif == 0):
        iterations = len(originalNames)
    elif(lengthdif > 0):
        iterations = len(originalNames)
    elif(lengthdif < 0):
        iterations = len(validatedNames)


    for x in range(iterations):
        skipRow = False
        skipRow2 = False
        #replace whitespace so we don't get weird scores
        originalNoWST, originalNoWSN, originalNoWSE, originalNoWSA = str(originalTitles[counterX]).replace(" ", ""), str(originalNames[counterX]).replace(" ", ""), str(originalEducation[counterX]).replace(" ", ""), str(originalAffiliation[counterX]).replace(" ", "")
        validatedNoWST, validatedNoWSN, validatedNoWSE, validatedNoWSA = str(validatedTitles[counterT]).replace(" ", ""), str(validatedNames[counterN]).replace(" ", ""), str(validatedEducation[counterE]).replace(" ", ""), str(validatedAffiliation[counterA]).replace(" ", "")
        distanceT, distanceN, distanceE, distanceA = lvs.ratio(originalNoWST, validatedNoWST), lvs.ratio(originalNoWSN, validatedNoWSN), lvs.ratio(originalNoWSE, validatedNoWSE), lvs.ratio(originalNoWSA, validatedNoWSA)

        #Name Distances
        if(distanceN < .6 and lengthdif > 0 or skipRow == True):
            distanceListN.append(0)
            counterN+=1
            skipRow = True 
        elif(distanceN < .6 and lengthdif < 0 or skipRow2 == True):
            distanceListN.append(0)
            skipRow2 = True

        #Title Distances
        if(skipRow == True):
            distanceListT.append(0)
            counterT+=1
        elif(skipRow2 == True):
            distanceListT.append(0)

    
        #Education Distances
        if(skipRow == True):
            distanceListE.append(0)
            counterE+=1
        elif(skipRow2 == True):
            distanceListE.append(0)

        #Affiliation distances
        if(skipRow == True):
            distanceListA.append(0)
            counterA+=1
        elif(skipRow2 == True):
            distanceListA.append(0)

        # #Schools distances
        # if(skipRow == True ):
        #     distanceListS.append(0)
        #     counterS+=1
        #     continue
        # elif(skipRow2 == True):
        #     distanceListS.append(0)
        #     continue

        distanceListN.append(distanceN)
        distanceListT.append(distanceT)
        distanceListE.append(distanceE)
        distanceListA.append(distanceA)
        # distanceListS.append(distanceS)

        #Increment the counters
        if(skipRow == False and skipRow2 == False):
            counterN, counterT, counterE, counterA, counterS, counterX = counterN+1, counterT+1, counterE+1, counterA+1, counterS+1, counterX+1
    
    #Compute average on all of them
    averageName, averageTitle, averageEducation, averageAffiliation = averageLevenshtein(distanceListN, 0), averageLevenshtein(distanceListT, 0), averageLevenshtein(distanceListE, 0), averageLevenshtein(distanceListA, 0)
    # print("Here is the average levenshtein distance for each column: ", averageName, averageTitle, averageEducation, averageAffiliation, averageSchool)
    if(weight == 0):
        fullAverageDistance = (averageName + averageTitle + averageEducation + averageAffiliation ) / 4
        return fullAverageDistance
    else:
        fullAverageDistance = (1.15*averageName + 0.88*averageTitle + 1.02*averageEducation + averageAffiliation )/4
        return fullAverageDistance 
    
    
#This function writes the average levenshtein to a text file
def writeDistances(averageDistance, pageName):
    path = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\SecondYearStuff\\ConnectedData\\ValidationSets\\refactoredDistances.txt"
    with open(path, 'a') as textfile:
        textfile.write(pageName + " Total Distances: " + str(averageDistance) + "\n")


def countMisses(original, validated):
    original = list(original.split("\n"))
    validated = list(validated.split("\n"))
    #find the larger list so we can choose which one to iterate until
    lengthdif = len(validated) - len(original)
    # print("This is length dif: ", lengthdif)
    return abs(lengthdif)
    

#This function takes the original names, and validated names, and calculates the levenshtein distance between all of them
def nameComparison(original, validated):
    distanceList = []
    original = list(original.split("\n"))
    validated = list(validated.split("\n"))
    #find the larger list so we can choose which one to iterate until
    lengthdif = len(validated) - len(original)
    if(lengthdif == 0):
        iterations = len(original)
    elif(lengthdif > 0):
        iterations = len(original)
    elif(lengthdif < 0):
        iterations = len(validated)

    counter = 0
    counterX = 0
    for x in range(iterations):
        #replace whitespace so we don't get weird scores
        originalNoWS = original[counterX].replace(" ", "")
        validatedNoWS = validated[counter].replace(" ", "")
        # print("These are the names: ", originalNoWS,   validatedNoWS)
        distance = lvs.ratio(originalNoWS, validatedNoWS)
        # print(originalNoWS, "     " , validatedNoWS)

        #If the names are way different, it's because there was a name that was missed, so catch it and increase/decrease the counter
        if(distance < .6 and lengthdif > 0):
            distanceList.append(0)
            counter+=1
            continue
        elif(distance < .6 and lengthdif < 0):
            distanceList.append(0)
            continue
        distanceList.append(distance)
        counter+=1
        counterX+=1

    # print(distanceList)
    return distanceList






# --------------- Method Testing --------------- #

# originalNames = read_names_original(filePathOriginal)
# validationNames = read_names_validated(filePathValidated)
# numMisses= countMisses(originalNames, validationNames)
# levDistances = nameComparison(originalNames, validationNames)
# averageDistance = averageLevenshtein(levDistances, 0)
# # writeDistances(numMisses, pageTitle)


# originalData = extractAllOriginal(filePathOriginal)
# validatedData = extractAllValidation(filePathValidated)
# fullDistances = tableCompare(originalData, validatedData,0)
# print(fullDistances)
# writeDistances(fullDistances, pageTitle)
# writeDistances(averageDistance, pageTitle)

# print("These are the original names: ", originalNames, "\n\n")
# print("These are the validation names: ", validationNames, "\n\n")
# print("Here are the individual Levenshtein distances of the names: ", levDistances)
# print("Here is the average Levenshtein distance of the names: ", averageDistance)

In [28]:
#Change to whatever you need
filePathOriginal = "C:\\Users\\tykun\\Downloads\\validationsets\\manualFormat\\winter2009_3_original.csv"
filePathValidated = "C:\\Users\\tykun\\Downloads\\validationsets\\manualFormat\\winter2009_3_validation.csv"
pageTitle = "winter2009_3_accuracies"



# --------------- Method Testing --------------- #

originalNames = read_names_validated(filePathOriginal)
validationNames = read_names_validated(filePathValidated)
numMisses= countMisses(originalNames, validationNames)
levDistances = nameComparison(originalNames, validationNames)
averageDistance = averageLevenshtein(levDistances, 0)
# writeDistances(numMisses, pageTitle)


originalData = extractAllValidation(filePathOriginal)
validatedData = extractAllValidation(filePathValidated)
fullDistances = tableCompare(originalData, validatedData,0)
print(fullDistances)
writeDistances(fullDistances, pageTitle)
writeDistances(averageDistance, pageTitle)

print("These are the original names: ", originalNames, "\n\n")
print("These are the validation names: ", validationNames, "\n\n")
print("Here are the individual Levenshtein distances of the names: ", levDistances)
print("Here is the average Levenshtein distance of the names: ", averageDistance)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tykun\\Downloads\\validationsets\\manualFormat\\winter2009_3_original.csv'